<a href="https://colab.research.google.com/github/jiyun-kong/Jeju_price_prediction_project/blob/main/%EC%A7%80%EC%9C%A4_jeju_AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Info.

1. train.csv
- train 데이터 : 2019년 01월 01일 ~ 2023년 03월 03일
  - item: TG : 감귤, BC : 브로콜리, RD : 무, CR : 당근, CB : 양배추
  - corporation : 유통 법인 코드
    - 법인 A ~ F 존재
  - location : 지역 코드
    - J : 제주도 제주시
    - S : 제주도 서귀포시
  - supply(kg) : 유통된 물량, kg 단위
  - price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위

2. international_trade.csv
- 관련 품목 수출입 정보

3. test.csv
- test 데이터 : 2023년 03월 04일 ~ 2023년 03월 31일

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.7 MB/s e

In [6]:
!pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7939 sha256=85ccb1b9e1992f8c8db9476599fca92683a2c44203824fcb07b60e687a3ed373
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [8]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [9]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [10]:
from pytimekr import pytimekr

year_2019 = pytimekr.holidays(year=2019)
year_2020 = pytimekr.holidays(year=2020)
year_2021 = pytimekr.holidays(year=2021)
year_2022 = pytimekr.holidays(year=2022)
year_2023 = pytimekr.holidays(year=2023)

def holidays(x):
    if x.weekday() in range(5,8):
        return 1
    if x.year == 2019  and x in year_2019 :
        return 1
    elif x.year == 2020 and x in year_2020:
        return 1
    elif x.year == 2021 and x in year_2021 :
        return 1
    elif x.year == 2022 and x in year_2022 :
        return 1
    elif x.year == 2023 and x in year_2023:
        return 1
    else:
        return 0

In [11]:
from sklearn.metrics import mean_squared_error

def rmse(y_test, y_pred):
  return np.sqrt(mean_squared_error(y_test, y_pred))

In [12]:
os.getcwd()

'/content'

In [25]:
train = pd.read_csv(os.getcwd() + "/drive/MyDrive/jeju_data/train.csv")
test = pd.read_csv(os.getcwd() + "/drive/MyDrive/jeju_data/test.csv")
trade = pd.read_csv(os.getcwd() + "/drive/MyDrive/jeju_data/international_trade.csv")
print(train.shape) # 총 59397개 데이터, 7개 컬럼
print(test.shape)
print(trade.shape)

(59397, 7)
(1092, 5)
(1274, 7)


In [14]:
print(len(train))
train['timestamp'].value_counts() # 39개씩 총 1523일

59397


2019-01-01    39
2021-10-09    39
2021-10-18    39
2021-10-17    39
2021-10-16    39
              ..
2020-05-20    39
2020-05-19    39
2020-05-18    39
2020-05-17    39
2023-03-03    39
Name: timestamp, Length: 1523, dtype: int64

In [15]:
test.head()

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J


In [16]:
from sklearn.preprocessing import LabelEncoder

def preprocess(data):
    data.rename(columns={'price(원/kg)':'price(won/kg)'}, inplace=True)

    # 날짜 데이터 -> 년도, 월, 일
    data['year'] = data['timestamp'].apply(lambda x : int(x[0:4]))
    data['month'] = data['timestamp'].apply(lambda x : int(x[5:7]))
    data['day'] = data['timestamp'].apply(lambda x : int(x[8:10]))

    # datetime 타입으로 변환
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # holiday
    data['holiday'] = data['timestamp'].apply(holidays)

    # timestamp 컬럼이 무조건 존재해야 함
    # ohe
    le = LabelEncoder()
    data['location'] = le.fit_transform(data['location'].values)
    data['corporation'] = le.fit_transform(data['corporation'].values)
    data['item'] = le.fit_transform(data['item'].values)

    return data

In [17]:
train_df = preprocess(train)
train_df['item_id'] = train_df.ID.str[0:6] # item_id 컬럼이 무조건 존재해야 함
train_df = train_df.drop(columns=['ID', 'supply(kg)'], axis=1)
print(train_df.shape)

(59397, 11)


In [26]:
test_df = preprocess(test)
test_df['item_id'] = test_df.ID.str[0:6]
test_df = test_df.drop(columns='ID')
print(test_df.shape)

(1092, 9)


In [19]:
train_df.head()

,timestamp,item,corporation,location,supply(kg),price(won/kg),year,month,day,holiday,item_id
0,2019-01-01,4,0,0,0.0,0.0,2019,1,1,1,TG_A_J
1,2019-01-02,4,0,0,0.0,0.0,2019,1,2,0,TG_A_J
2,2019-01-03,4,0,0,60601.0,1728.0,2019,1,3,0,TG_A_J
3,2019-01-04,4,0,0,25000.0,1408.0,2019,1,4,0,TG_A_J
4,2019-01-05,4,0,0,32352.0,1250.0,2019,1,5,1,TG_A_J


In [27]:
test_df.head()

,timestamp,item,corporation,location,year,month,day,holiday,item_id
0,2023-03-04,4,0,0,2023,3,4,1,TG_A_J
1,2023-03-05,4,0,0,2023,3,5,1,TG_A_J
2,2023-03-06,4,0,0,2023,3,6,0,TG_A_J
3,2023-03-07,4,0,0,2023,3,7,0,TG_A_J
4,2023-03-08,4,0,0,2023,3,8,0,TG_A_J


# AutoGluon

In [21]:
data = TimeSeriesDataFrame(train_df)
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target='price(won/kg)',
    eval_metric="RMSE",
)
# seed 고정
predictor.fit(data, random_seed=42)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(won/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231114_023439/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(won/kg)'
	past covariates:  ['item', 'c

In [22]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	9.05    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 9.15 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [23]:
pred = predictor.predict(data, random_seed=42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [30]:
submission = pd.read_csv(os.getcwd() + "/drive/MyDrive/jeju_data/sample_submission.csv")
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv(os.getcwd() + '/dacon_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3267.430632
1,TG_A_J_20230305,613.399876
2,TG_A_J_20230306,3062.702166
3,TG_A_J_20230307,3457.144647
4,TG_A_J_20230308,3357.136627
...,...,...
1087,RD_F_J_20230327,492.997841
1088,RD_F_J_20230328,516.488705
1089,RD_F_J_20230329,514.444653
1090,RD_F_J_20230330,491.955314
